In [0]:
%sql
-- Created a new schema for the project 
-- This keeps all project tables organized in one place
CREATE SCHEMA IF NOT EXISTS ecommerce_2024

In [0]:
%sql
-- Set ecommerce_2024 as the active schema
-- All tables created after this will live here by default
USE ecommerce_2024;

In [0]:
%sql
-- Show all tables in the current schema
-- At this point, it should be empty
SHOW TABLES

database,tableName,isTemporary


In [0]:
%sql
-- Count how many rows exist in the source table
-- This gives an idea of how many rows we have in total
SELECT COUNT(*) AS total_rows
FROM src_orders_2024;

total_rows
1000


In [0]:
%sql
-- Minimum and maximum order dates in the source table
SELECT
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date
FROM src_orders_2024;

min_order_date,max_order_date
2024-01-01,2024-12-30


In [0]:
%sql
-- Create the raw_orders table to store raw data in Delta format
-- This is the Bronze layer: minimal changes, just structured as-is
CREATE TABLE IF NOT EXISTS raw_orders (
  row_id BIGINT,
  customer_id STRING,
  order_date DATE,
  order_id STRING,
  sales DOUBLE
)
USING DELTA;

In [0]:
%sql
-- Verify that the tables now exist in the schema
SHOW TABLES;

database,tableName,isTemporary
ecommerce_2024,raw_orders,false
ecommerce_2024,src_orders_2024,false


In [0]:
%sql
-- Insert data from the source table into the Bronze table
-- Apply very light cleaning:
-- Trim spaces from text fields
-- Cast sales to DOUBLE to enforce numeric type
INSERT INTO raw_orders
SELECT
  row_id,
  TRIM(customer_id) AS customer_id,
  order_date,
  TRIM(order_id) AS order_id,
  CAST(sales AS DOUBLE) AS sales
FROM src_orders_2024;

num_affected_rows,num_inserted_rows
1000,1000


In [0]:
%sql
-- total rows
SELECT COUNT(*) FROM raw_orders;

COUNT(*)
1000


In [0]:
%sql
-- unique orders
SELECT COUNT(DISTINCT order_id) FROM raw_orders;


COUNT(DISTINCTorder_id)
1000


In [0]:
%sql
-- unique customers
SELECT COUNT(DISTINCT customer_id) FROM raw_orders;

COUNT(DISTINCTcustomer_id)
649


In [0]:
%sql
-- date range
SELECT MIN(order_date), MAX(order_date) FROM raw_orders;

MIN(order_date),MAX(order_date)
2024-01-01,2024-12-30


In [0]:
%sql
-- Create (or replace) the Silver table clean_orders as a Delta table
-- If the table already exists, it will be overwritten with fresh data
CREATE OR REPLACE TABLE clean_orders
USING DELTA
AS

-- First CTE: filter out invalid rows
-- Silver is where bad data gets removed
-- We'll keep only rows with valid values for all fields
WITH filtered AS (
  SELECT
    row_id,
    customer_id,
    order_date,
    order_id,
    sales
  FROM raw_orders
  WHERE customer_id IS NOT NULL --remove rows with missing customer IDs
    AND order_id IS NOT NULL    --remove rows with missing order IDs
    AND order_date IS NOT NULL  --remove rows with missing order dates
    AND sales IS NOT NULL       --remove rows with missing sales values
    AND sales > 0               --remove rows with zero or negative sales
),


-- Second CTE: deduplicate orders
-- Because some orders may appear more than once, we keep only the most recent order_date
-- If tied, the one with the highest row_id
deduped AS (
  SELECT
    *,
  -- Give each row a number within the same order_id
   -- Row number 1 means: "this is the row we want to keep"
   -- We first look at the newest order_date
   -- If two rows have the same date, we use row_id to decide
    ROW_NUMBER() OVER (
      PARTITION BY order_id
      ORDER BY order_date DESC, row_id DESC
    ) AS rn
  FROM filtered
)

-- Final SELECT: keep only one row per order_id
SELECT
  row_id,
  customer_id,
  order_date,
  order_id,
  sales
FROM deduped
-- keep only the first (best) row per order_id
WHERE rn = 1;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- -- Verify that the tables now exist in the schema
SHOW TABLES;


database,tableName,isTemporary
ecommerce_2024,clean_orders,false
ecommerce_2024,raw_orders,false
ecommerce_2024,src_orders_2024,false


In [0]:
%sql
-- total rows
SELECT COUNT(*) AS total_rows
FROM clean_orders;

total_rows
1000


In [0]:
%sql
-- unique orders (this MUST equal total_rows)
SELECT COUNT(DISTINCT order_id) AS unique_orders
FROM clean_orders;


unique_orders
1000


In [0]:
%sql
-- unique customers
SELECT COUNT(DISTINCT customer_id) AS unique_customers
FROM clean_orders;

unique_customers
649


In [0]:
%sql
-- date range
SELECT
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date
FROM clean_orders;

min_order_date,max_order_date
2024-01-01,2024-12-30


In [0]:
%sql
-- Create a GOLD table that summarizes the customer cohort behaviour
-- One row per customer: first purchase date, second purchase(if any), cohort month(month of first purchase) and days between first and second purchase
CREATE OR REPLACE TABLE cohort_customers
USING DELTA
AS

-- Step 1: Order each customer's purchases from oldest to newest
-- We assign a rank number to each order:
-- rn = 1 means "this customer's first order"
-- rn = 2 means "this customer's second order"
WITH ordered AS (
  SELECT
    customer_id,
    order_date,
    -- Rank each customer's orders by date (earliest order gets rn = 1)
    ROW_NUMBER() OVER (
      PARTITION BY customer_id
      ORDER BY order_date ASC
    ) AS rn
  FROM clean_orders
),

-- Step 2: For each customer, pick their 1st and 2nd purchase dates
-- We use MIN(...) + CASE to pull specific ranked dates into columns
first_second AS (
  SELECT
    customer_id,
    -- First purchase date = the earliest order (rn=1)
    MIN(CASE WHEN rn = 1 THEN order_date END) AS first_purchase_date,
    -- Second purchase date = the next earliest order (rn=2) (if it exists)
    MIN(CASE WHEN rn = 2 THEN order_date END) AS second_purchase_date
  FROM ordered
  GROUP BY customer_id
)


-- Step 3: Build the final cohort table
SELECT
  customer_id,
  first_purchase_date,
  second_purchase_date,
  -- Month of the first purchase (cohort)
  -- (this is how we group customers into cohorts)
  DATE_TRUNC('month', first_purchase_date) AS cohort_month,
  -- Days between first and second purchase (null if no second purchase)
  DATEDIFF(second_purchase_date, first_purchase_date) AS days_to_second_purchase
FROM first_second;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Verify that the tables now exist in the schema
SHOW TABLES;

database,tableName,isTemporary
ecommerce_2024,clean_orders,false
ecommerce_2024,cohort_customers,false
ecommerce_2024,raw_orders,false
ecommerce_2024,src_orders_2024,false


In [0]:
%sql
-- total customers (should equal distinct customers in clean_orders)
SELECT COUNT(*) AS total_customers
FROM cohort_customers;

total_customers
649


In [0]:
%sql
-- repeaters vs one-time customers
SELECT
  SUM(CASE WHEN second_purchase_date IS NOT NULL THEN 1 ELSE 0 END) AS repeat_customers,
  SUM(CASE WHEN second_purchase_date IS NULL THEN 1 ELSE 0 END) AS one_time_customers
FROM cohort_customers;

repeat_customers,one_time_customers
184,465


In [0]:
%sql
-- cohort months range
SELECT
  MIN(cohort_month) AS min_cohort,
  MAX(cohort_month) AS max_cohort
FROM cohort_customers;

min_cohort,max_cohort
2024-01-01T00:00:00.000Z,2024-12-01T00:00:00.000Z


In [0]:
%sql
SELECT *
FROM ecommerce_2024.raw_orders
LIMIT 100;

row_id,customer_id,order_date,order_id,sales
241,CUST546,2024-01-01,ORD1240,157.06
515,CUST260,2024-01-01,ORD1514,79.44
239,CUST303,2024-01-01,ORD1238,261.57
214,CUST507,2024-01-01,ORD1213,177.22
823,CUST411,2024-01-01,ORD1822,140.94
26,CUST266,2024-01-02,ORD1025,34.46
566,CUST322,2024-01-02,ORD1565,240.26
27,CUST522,2024-01-02,ORD1026,189.66
713,CUST467,2024-01-02,ORD1712,179.93
382,CUST648,2024-01-02,ORD1381,164.95


In [0]:
%sql
SELECT *
FROM ecommerce_2024.clean_orders
LIMIT 100;

row_id,customer_id,order_date,order_id,sales
1,CUST272,2024-05-17,ORD1000,296.26
2,CUST151,2024-01-02,ORD1001,286.33
3,CUST554,2024-09-18,ORD1002,53.17
4,CUST061,2024-04-08,ORD1003,198.38
5,CUST578,2024-11-06,ORD1004,54.14
6,CUST188,2024-06-23,ORD1005,212.11
7,CUST670,2024-08-02,ORD1006,23.44
8,CUST353,2024-02-04,ORD1007,251.11
9,CUST039,2024-06-14,ORD1008,102.7
10,CUST253,2024-12-19,ORD1009,143.85


In [0]:
%sql
SELECT *
FROM ecommerce_2024.cohort_customers
LIMIT 100;

customer_id,first_purchase_date,second_purchase_date,cohort_month,days_to_second_purchase
CUST001,2024-04-27,null,2024-04-01T00:00:00.000Z,null
CUST002,2024-06-05,2024-10-31,2024-06-01T00:00:00.000Z,148
CUST003,2024-07-25,2024-10-13,2024-07-01T00:00:00.000Z,80
CUST004,2024-09-24,null,2024-09-01T00:00:00.000Z,null
CUST007,2024-02-14,null,2024-02-01T00:00:00.000Z,null
CUST008,2024-02-17,2024-11-20,2024-02-01T00:00:00.000Z,277
CUST010,2024-11-05,2024-12-10,2024-11-01T00:00:00.000Z,35
CUST011,2024-10-10,null,2024-10-01T00:00:00.000Z,null
CUST012,2024-03-14,null,2024-03-01T00:00:00.000Z,null
CUST013,2024-02-23,2024-03-19,2024-02-01T00:00:00.000Z,25


In [0]:
%sql
-- How many customers have no second purchase
SELECT
  COUNT(*) AS customers_total,
  SUM(CASE WHEN second_purchase_date IS NULL THEN 1 ELSE 0 END) AS customers_no_second_purchase
FROM ecommerce_2024.cohort_customers;


customers_total,customers_no_second_purchase
649,465


In [0]:
%sql
-- Validate a NULL case by choosing one customer with NULL second purchase and confirm they only have 1 order
SELECT
  customer_id,
  COUNT(DISTINCT order_id) AS orders
FROM ecommerce_2024.clean_orders
WHERE customer_id = 'CUST001'   -- taken from cohort_customers table
GROUP BY customer_id;


customer_id,orders
CUST001,1


In [0]:
%sql
SELECT COUNT(*) FROM ecommerce_2024.src_orders_2025;


COUNT(*)
1000


In [0]:
%sql
SELECT
  MIN(order_date),
  MAX(order_date)
FROM ecommerce_2024.src_orders_2025;


MIN(order_date),MAX(order_date)
2025-01-01,2025-12-31


In [0]:
%sql
-- Append the 2025 rows into the raw_orders table using same formatting rules as before

INSERT INTO ecommerce_2024.raw_orders
SELECT
  row_id,
  TRIM(customer_id) AS customer_id,
  order_date,
  TRIM(order_id) AS order_id,
  CAST(sales AS DOUBLE) AS sales
FROM ecommerce_2024.src_orders_2025;


num_affected_rows,num_inserted_rows
1000,1000


In [0]:
%sql
-- Checking if min_date belongs to 2024, max_date to 2025
SELECT
  MIN(order_date) AS min_date,
  MAX(order_date) AS max_date,
  COUNT(*) AS total_rows
FROM ecommerce_2024.raw_orders;


min_date,max_date,total_rows
2024-01-01,2025-12-31,2000


In [0]:
%sql
--Rebuild clean_orders so it reflects the full dataset (2024+2025) using the same logic as before
CREATE OR REPLACE TABLE ecommerce_2024.clean_orders
USING DELTA
AS
WITH filtered AS (
  SELECT
    row_id,
    customer_id,
    order_date,
    order_id,
    sales
  FROM ecommerce_2024.raw_orders
  WHERE customer_id IS NOT NULL
    AND order_id IS NOT NULL
    AND order_date IS NOT NULL
    AND sales IS NOT NULL
    AND sales > 0
),
deduped AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY order_id
      ORDER BY order_date DESC, row_id DESC
    ) AS rn
  FROM filtered
)
SELECT
  row_id,
  customer_id,
  order_date,
  order_id,
  sales
FROM deduped
WHERE rn = 1;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Checking if min_date is still 2024, max_date in 2025
-- Also checking if total rows is equal to distinct_orders 

SELECT
  MIN(order_date) AS min_date,
  MAX(order_date) AS max_date,
  COUNT(*) AS total_rows,
  COUNT(DISTINCT order_id) AS distinct_orders
FROM ecommerce_2024.clean_orders;


min_date,max_date,total_rows,distinct_orders
2024-01-01,2025-12-31,2000,2000


In [0]:
%sql
-- Rebuild cohort_customers from the previously updated clean_orders
CREATE OR REPLACE TABLE ecommerce_2024.cohort_customers
USING DELTA
AS
WITH ranked_orders AS (
  SELECT
    customer_id,
    order_date,
    ROW_NUMBER() OVER (
      PARTITION BY customer_id
      ORDER BY order_date
    ) AS purchase_rank
  FROM ecommerce_2024.clean_orders
),

first_second AS (
  SELECT
    customer_id,
    MIN(CASE WHEN purchase_rank = 1 THEN order_date END) AS first_purchase_date,
    MIN(CASE WHEN purchase_rank = 2 THEN order_date END) AS second_purchase_date
  FROM ranked_orders
  GROUP BY customer_id
)

SELECT
  customer_id,
  first_purchase_date,
  second_purchase_date,
  DATE_TRUNC('month', first_purchase_date) AS cohort_month,
  DATEDIFF(second_purchase_date, first_purchase_date) AS days_to_second_purchase
FROM first_second;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Checking if cohort_month now includes 2025
SELECT
  MIN(cohort_month) AS min_cohort_month,
  MAX(cohort_month) AS max_cohort_month,
  COUNT(*) AS total_customers
FROM ecommerce_2024.cohort_customers;



min_cohort_month,max_cohort_month,total_customers
2024-01-01T00:00:00.000Z,2025-12-01T00:00:00.000Z,888


In [0]:
%sql
-- Checking how many customers have no second purchase
SELECT
  COUNT(*) AS customers_total,
  SUM(CASE WHEN second_purchase_date IS NULL THEN 1 ELSE 0 END) AS customers_no_second_purchase
FROM ecommerce_2024.cohort_customers;



customers_total,customers_no_second_purchase
888,308


In [0]:
%sql
-- Reporting queries used in the dashboard

-- Cohort size
SELECT
  cohort_month,
  COUNT(DISTINCT customer_id) AS new_customers
FROM ecommerce_2024.cohort_customers
GROUP BY cohort_month;

-- Retention to 2nd purchase
SELECT
  cohort_month,
  COUNT(*) AS cohort_size,

  SUM(CASE WHEN days_to_second_purchase <= 30 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS retention_1_month,
  SUM(CASE WHEN days_to_second_purchase <= 60 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS retention_2_months,
  SUM(CASE WHEN days_to_second_purchase <= 90 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS retention_3_months

FROM ecommerce_2024.cohort_customers
GROUP BY cohort_month
ORDER BY cohort_month;

-- Repeat purchase depth
WITH orders_per_customer AS (
  SELECT
    c.customer_id,
    c.cohort_month,
    COUNT(DISTINCT o.order_id) AS total_orders
  FROM ecommerce_2024.cohort_customers c
  JOIN ecommerce_2024.clean_orders o
    ON c.customer_id = o.customer_id
  GROUP BY c.customer_id, c.cohort_month
)

SELECT
  cohort_month,
  COUNT(*) AS cohort_size,
  SUM(CASE WHEN total_orders >= 2 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS repeat_2plus,
  SUM(CASE WHEN total_orders >= 3 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS repeat_3plus,
  SUM(CASE WHEN total_orders >= 4 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS repeat_4plus
FROM orders_per_customer
GROUP BY cohort_month
ORDER BY cohort_month;


  -- When created the dashboards for 2024 dataset 
  -- I used this condition for the year filter
  -- WHERE cohort_month >= '2024-01-01' AND cohort_month < '2025-01-01'
  -- So dashboard 1 has the 2024 filter, Dashboard 2 has no filter (so it includes both 2024 and 2025)
